# Laboratorio 8    
## Inteligencia Artificial

**Integrantes**
- Gabriel García    - 21352     
- Diego Leiva       - 21752   
- Pablo Orellana    - xd

----

## Task 1: Teoría
1. Investigar el algoritmo AC-3 y su relacion con el algoritmo de backtracking search       

El algormitmo AC-3 (Algoritmo de conssitencia de Arcos-3) Se utiliza en resolucion de problemas de restricción para reducir el dominio de las variables. Se centra en eliminar los valores incosistentes de las variables basandose en restricciones binarias. AC-3 se relaciona con el algoritmo de backtracking search en el sentido de que AC-3 se utiliza a menudo con el preprocesamiento para reducir el espacio de busqueda antes de aplicar backtracking search. AC-3 ayuda a identificar y eliminar las inconsistencias temprano, lo que puede reducir significativamente el tiempo de busqueda en el algoritmo de backtracking (GeeksforGeeks, 2023).     

2. Defina en sus propias palabras el termino "Arc Consistency"     

Arc Consistency se refiere a la propiedad en un problema de restricción donde los valores en el dominio de una variable son consistentes con las restricciones binarias que tienen con otras variables. Es decir, un arco (una conexion entre dos variables) se considera consistente si para el valor en el dominio de la variable A, hay al menos un valor en el dominio de la variable B que satisface la restricción binaria entre A y B. Por lo que podemos concluir que "Arc Consistency" es una medida de que tan restringido es el dominio de una variable basándose en las restricciones binarias que tienen con otras variables en un problema de restricción (British Columbia University, 2009).  



Referencias
- GeeksforGeeks. (2023, 20 de octubre). Backtracking Algorithms. GeeksforGeeks. https://www.geeksforgeeks.org/backtracking-algorithms/

- British Columbia University. (2009, 10). CSP3: Backtracking Search. Department of Computer Science. https://www.cs.ubc.ca/~kevinlb/teaching/cs322%20-%202009-10/Lectures/CSP3.pdf

----

## Task 2: CSP con Backtracking, Beam y Local Search

**Importar librerias**

In [1]:
import pandas as pd
import random
import itertools
import time

**Definicion del Problema**

Calendarizacion de exámenes para cuatro estudiantes que toman siete exámenes diferentes. El problema implica calendarizar los exámenes para los estudiantes respetando diversas limitaciones y preferencias.

Restricciones:
- Todos los exámenes deberán realizarse en días diferentes, concretamente lunes, martes y miércoles.
- Ningún estudiante deberá tener más de un examen por día.
- Los estudiantes que toman el mismo curso no pueden tener exámenes programados para el mismo día.

In [2]:
# Datos del problema

estudiantes = ['Estudiante 1', 'Estudiante 2', 'Estudiante 3', 'Estudiante 4']              # Lista de estudiantes
dias = ['Lunes', 'Martes', 'Miercoles']                                                     # Lista de días                         
cursos = ['Curso 1', 'Curso 2', 'Curso 3', 'Curso 4', 'Curso 5', 'Curso 6', 'Curso 7']      # Lista de los examenes

### Algoritmo Backtrack

In [3]:
# Crear las variables y dominios
variables = [(student, day) for student in estudiantes for day in dias]
dominios = {var: cursos for var in variables}  # Todos los cursos están disponibles para todos inicialmente

# Funciones para el algoritmo Backtracking
def backtrack(assignment):
    if len(assignment) == len(variables):
        return assignment

    var = select_unassigned_variable(assignment)
    for value in order_domain_values(var, assignment):
        if is_consistent(var, value, assignment):
            assignment[var] = value
            result = backtrack(assignment)
            if result is not None:
                return result
            del assignment[var]
    return None

def select_unassigned_variable(assignment):
    for var in variables:
        if var not in assignment:
            return var

def order_domain_values(var, assignment):
    student, _ = var
    taken_courses = [value for (s, _), value in assignment.items() if s == student]
    return [course for course in dominios[var] if course not in taken_courses]

def is_consistent(var, value, assignment):
    student, day = var
    for (other_student, other_day), other_course in assignment.items():
        if other_day == day and other_course == value and student != other_student:
            return False
    return True

# Resolver el problema CSP
asignacion_backtrack = {}
solucion_backtrack = backtrack(asignacion_backtrack)

# Crear y llenar la matriz de horarios
backtrack_matrix = pd.DataFrame(index=estudiantes, columns=dias)
if solucion_backtrack:
    for (student, day), course in solucion_backtrack.items():
        backtrack_matrix.at[student, day] = course

# Imprimir la matriz de horarios
backtrack_matrix

,Lunes,Martes,Miercoles
Estudiante 1,Curso 1,Curso 2,Curso 3
Estudiante 2,Curso 2,Curso 1,Curso 4
Estudiante 3,Curso 3,Curso 4,Curso 1
Estudiante 4,Curso 4,Curso 3,Curso 2


### Algoritmo Beam Search

In [4]:
# Inicializar una asignación aleatoria
def inicializar_asignacion():
    asignacion = {}
    for estudiante in estudiantes:
        asignacion[estudiante] = random.sample(cursos, 3)
    return asignacion


# Función para verificar si una asignación es válida
def es_valida(asignacion):
    # Comprobar si dos estudiantes tienen el mismo examen el mismo día
    for dia_idx in range(3):  # 0 para lunes, 1 para martes, 2 para miércoles
        dia_cursos = [asignacion[est][dia_idx] for est in asignacion]
        if len(dia_cursos) != len(set(dia_cursos)):  # Si hay duplicados, no es válida
            return False
    return True

# Generar estados sucesores
def generar_sucesores(asignacion):
    sucesores = []
    for estudiante, cursos_asignados in asignacion.items():
        for dia_idx in range(3):
            for curso in cursos:
                # Crear un nuevo estado cambiando el curso de un día
                nuevo_estado = {e: c[:] for e, c in asignacion.items()}  # Copiar el estado
                if curso not in cursos_asignados:  # Evitar asignar el mismo curso dos veces al mismo estudiante
                    nuevo_estado[estudiante][dia_idx] = curso
                    if es_valida(nuevo_estado):
                        sucesores.append(nuevo_estado)
    return sucesores

# Función de evaluación: contar el número de conflictos
def evaluar(asignacion):
    conflictos = 0
    for dia_idx in range(3):
        dia_cursos = [asignacion[est][dia_idx] for est in asignacion]
        conflictos += len(dia_cursos) - len(set(dia_cursos))  # Contar duplicados
    return conflictos

# Beam Search
def beam_search(k):
    estado_actual = [inicializar_asignacion() for _ in range(k)]
    while True:
        todos_sucesores = list(itertools.chain(*[generar_sucesores(est) for est in estado_actual]))
        if not todos_sucesores:
            break
        estado_actual = sorted(todos_sucesores, key=lambda est: evaluar(est))[:k]
        if evaluar(estado_actual[0]) == 0:
            return estado_actual[0]
    return estado_actual[0] if estado_actual else None


# Ejecutar beam search
solucion_beam = beam_search(4)  # k es el ancho del haz


# Crear un DataFrame para representar la asignación
beam_matrix = pd.DataFrame(solucion_beam).T  # Transponemos para tener a los estudiantes como filas
beam_matrix.columns = dias  # Establecer los días como columnas

# Imprimir la solución
beam_matrix

,Lunes,Martes,Miercoles
Estudiante 1,Curso 5,Curso 2,Curso 6
Estudiante 2,Curso 7,Curso 2,Curso 6
Estudiante 3,Curso 3,Curso 4,Curso 1
Estudiante 4,Curso 6,Curso 4,Curso 5


### Algoritmo Local Search

In [5]:
# Inicialización aleatoria de la asignación de exámenes
asignacion_local = {estudiante: random.sample(cursos, len(dias)) for estudiante in estudiantes}

def contar_conflictos(asignacion):
    conflictos = 0
    # Verificar conflictos entre estudiantes para cada día
    for dia in range(len(dias)):
        cursos_dia = [asignacion[estudiante][dia] for estudiante in estudiantes]
        conflictos += len(cursos_dia) - len(set(cursos_dia))  # Cursos duplicados indican conflictos
    return conflictos

def local_search(asignacion):
    iteraciones = 0
    while True:
        iteraciones += 1
        conflictos_actual = contar_conflictos(asignacion)
        if conflictos_actual == 0:
            break  # Salir si no hay conflictos

        # Intentar un cambio aleatorio para reducir los conflictos
        estudiante_aleatorio = random.choice(estudiantes)
        dia_aleatorio = random.randint(0, len(dias) - 1)
        curso_actual = asignacion[estudiante_aleatorio][dia_aleatorio]
        nuevo_curso = random.choice([curso for curso in cursos if curso not in asignacion[estudiante_aleatorio]])
        asignacion[estudiante_aleatorio][dia_aleatorio] = nuevo_curso

        # Evaluar si el cambio redujo los conflictos
        nuevos_conflictos = contar_conflictos(asignacion)
        if nuevos_conflictos > conflictos_actual:
            # Revertir el cambio si no se redujeron los conflictos
            asignacion[estudiante_aleatorio][dia_aleatorio] = curso_actual
        
        # Limitar el número de iteraciones para evitar un bucle infinito
        if iteraciones > 1000:
            break

    return asignacion, iteraciones

asignacion_final, iteraciones = local_search(asignacion_local)

# Convertir la asignación final a una estructura de datos adecuada para crear un DataFrame
data = {dia: [asignacion_final[estudiante][i] for estudiante in estudiantes] for i, dia in enumerate(dias)}

# Crear el DataFrame usando Pandas
local_matrix = pd.DataFrame(data, index=estudiantes)

local_matrix

,Lunes,Martes,Miercoles
Estudiante 1,Curso 4,Curso 1,Curso 6
Estudiante 2,Curso 1,Curso 7,Curso 3
Estudiante 3,Curso 3,Curso 4,Curso 5
Estudiante 4,Curso 5,Curso 3,Curso 7


## Rendimiento de los algoritmos

**Medicion de tiempo de ejecucion**

In [6]:
def profile_function(funcion, *args, n=1000):
    tiempos = []
    for _ in range(n):
        inicio = time.time()
        funcion(*args)
        fin = time.time()
        tiempos.append(fin - inicio)
    return tiempos

**Obtener tiempos promedio de ejecucion**

In [7]:
def analizar_tiempos(metodo, tiempos):
    promedio = sum(tiempos) / len(tiempos)
    print(f"Tiempo promedio de ejecución para {metodo}: {promedio} segundos.")

**Evaluando Backtracking**

In [11]:
tiempos_backtrack = profile_function(backtrack, {})
analizar_tiempos('Backtrack', tiempos_backtrack)

Tiempo promedio de ejecución para Backtrack: 1.0001659393310548e-06 segundos.


**Evaluando Beam Search**

In [9]:
tiempos_beam_search = profile_function(beam_search, 4)
analizar_tiempos('Beam Search', tiempos_beam_search)

Tiempo promedio de ejecución para Beam Search: 0.00032730770111083984 segundos.


**Evaluando Local Search**

In [10]:
def ejecutar_local_search():
    asignacion_local = {estudiante: random.sample(cursos, len(dias)) for estudiante in estudiantes}
    return local_search(asignacion_local)

tiempos_local_search = profile_function(ejecutar_local_search)
analizar_tiempos('Local Search', tiempos_local_search)

Tiempo promedio de ejecución para Local Search: 5.644416809082031e-05 segundos.


## Análisis comparativo de algoritmos

**Resultados de tiempos de ejecucion**

- Backtrack: Tiempo promedio de ejecución 1.000×10E−06 segundos.
- Beam Search: Tiempo promedio de ejecución 3.273×10E−04 segundos.
- Local Search: Tiempo promedio de ejecución 5.644×10E−05 segundos.

**Comparación entre algoritmos**

El algoritmo Backtrack demostró ser el más rápido, lo cual es esperado en escenarios donde el espacio de búsqueda no es excesivamente grande, ya que explora exhaustivamente todas las posibilidades hasta encontrar una solución.
Beam Search, siendo una búsqueda heurística, mostró tiempos mayores comparado con Backtrack. Esto puede deberse a la exploración parcial del espacio de búsqueda que realiza, guiada por la heurística que prioriza ciertos nodos.
Local Search tuvo un rendimiento intermedio, lo cual sugiere que para este problema específico y la configuración dada, encontró soluciones de manera eficiente pero sin ser la más rápida. Esto puede ser debido a la naturaleza estocástica del algoritmo y su dependencia de la calidad de la asignación inicial.

Por otro lado en cuanto a las soluciones encontradas Backtracking obtuvo en todas sus iteraciones la misma solución, mientras que los otros dos algoritmos (beam y local search) encontraron mas de una posible solución al problema.
En todos los casos las soluciones encontradas cumplen con las restricciones impuestas al principio en el problema.

**Conclusiones**
- Backtrack es altamente efectivo para espacios de búsqueda limitados pero puede no escalar bien con problemas más grandes.
- Beam Search ofrece un compromiso entre calidad de solución y tiempo de ejecución, siendo más adecuado para problemas donde una solución exacta no es crítica.
- Local Search es útil para espacios de búsqueda grandes donde una solución aproximada es aceptable, aunque su rendimiento puede variar significativamente dependiendo de la asignación inicial.